<a href="https://colab.research.google.com/github/lokeshkumarn/Notebooks/blob/master/DeepSpeechv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)

In [0]:
#@title
#https://github.com/mayukhnair/deepspeech-colab/blob/master/deepspeech-colab.ipynb
!wget -O git-lfs.tar.gz https://github.com/git-lfs/git-lfs/releases/download/v2.4.2/git-lfs-linux-amd64-2.4.2.tar.gz
!tar -xvzf git-lfs.tar.gz
!./git-lfs-2.4.2/install.sh
# %cd "/content/drive/My Drive/"                                        
# !pwd
!git lfs clone https://github.com/mozilla/deepspeech
%cd "/content/deepspeech"
!pip install -r requirements.txt
!pip uninstall -y tensorflow
!pip uninstall -y tensorflow-gpu
!pip install deepspeech-gpu
#https://www.tensorflow.org/install/gpu
#CUDA® Toolkit —TensorFlow supports CUDA 10.0 (TensorFlow >= 1.13.0)
!pip install tensorflow-gpu==1.14.0

In [0]:
restart_runtime()

In [0]:
%cd "/content/deepspeech"

**Fetching the pre-trained model & sample audio clip**

In [0]:
#!wget -O - https://github.com/mozilla/DeepSpeech/releases/download/v0.1.1/deepspeech-0.1.1-models.tar.gz | tar xvfz -
#!wget -O - https://github.com/mozilla/DeepSpeech/releases/download/v0.5.1/deepspeech-0.5.1-models.tar.gz | tar xvfz -
!wget -O - https://github.com/mozilla/DeepSpeech/releases/download/v0.6.0/deepspeech-0.6.0-models.tar.gz | tar xvfz -
#!wget -O - https://github.com/mozilla/DeepSpeech/releases/download/v0.1.1/deepspeech-0.1.1-checkpoint.tar.gz | tar xvfz -
#!wget -O - https://github.com/mozilla/DeepSpeech/releases/download/v0.5.1/deepspeech-0.5.1-checkpoint.tar.gz | tar xvfz -
!wget -O - https://github.com/mozilla/DeepSpeech/releases/download/v0.6.0/deepspeech-0.6.0-checkpoint.tar.gz | tar xvfz -
!mkdir customdata                              
!wget -O customdata/sample-1.wav https://github.com/mayukhnair/dsdatasets/raw/master/sample-1.wav
!wget -O customdata/ttssample1.wav https://github.com/mayukhnair/dsdatasets/raw/master/ttssample1.wav

**CUDA Setup in Colab**

In [0]:
!uname -a

In [0]:
# Add NVIDIA package repositories
# Add HTTPS support for apt-key
!apt-get install gnupg-curl
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_10.0.130-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604_10.0.130-1_amd64.deb
!apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub
!apt-get update
!wget http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/nvidia-machine-learning-repo-ubuntu1604_1.0.0-1_amd64.deb
!apt install ./nvidia-machine-learning-repo-ubuntu1604_1.0.0-1_amd64.deb
!apt-get update

# Install NVIDIA driver
# Issue with driver install requires creating /usr/lib/nvidia
!mkdir /usr/lib/nvidia
!apt-get install --no-install-recommends nvidia-418
# Reboot. Check that GPUs are visible using the command: nvidia-smi

# Install development and runtime libraries (~4GB)
!apt-get install --no-install-recommends cuda-10-0 libcudnn7=7.6.2.24-1+cuda10.0 libcudnn7-dev=7.6.2.24-1+cuda10.0


# Install TensorRT. Requires that libcudnn7 is installed above.
!apt-get install -y --no-install-recommends libnvinfer5=5.1.5-1+cuda10.0 libnvinfer-dev=5.1.5-1+cuda10.0

In [0]:
#https://developer.nvidia.com/cuda-downloads?target_os=Linux&target_arch=x86_64&target_distro=Ubuntu&target_version=1604&target_type=deblocal
#https://www.tensorflow.org/install/gpu
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-ubuntu1604.pin
!mv cuda-ubuntu1604.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget http://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda-repo-ubuntu1604-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
!apt-key add /var/cuda-repo-10-2-local-10.2.89-440.33.01/7fa2af80.pub
!apt-get update
!apt-get -y install cuda

**GPU Training Setup**

In [0]:
#OLD
#!python "/content/drive/My Drive/DeepSpeech/util/taskcluster.py" --target "/content/drive/My Drive/DeepSpeech/native_client_new"
#!apt-get install dirmngr
# !wget -O cuda9base.deb https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
# !dpkg -i cuda9base.deb
# !apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
# #!apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1704/x86_64/7fa2af80.pub
# !apt-get update
# !apt-get install cuda=9.0.176-1

In [0]:
!export LD_LIBRARY_PATH=/usr/local/cuda-9.0/lib64:$LD_LIBRARY_PATH

In [0]:
!apt install nvidia-cuda-toolkit

In [0]:
!wget -O cudnn7.tgz https://www.dropbox.com/s/dgmc6extwj8u14m/cudnn-9.0-linux-x64-v7.tgz
!cp -P cuda/include/cudnn.h /usr/include
!cp -P cuda/lib64/libcudnn* /usr/lib/x86_64-linux-gnu/
#Permission
!chmod a+r /usr/lib/x86_64-linux-gnu/libcudnn*
!cp cuda/include/cudnn.h /usr/local/cuda/include
!cp cuda/lib64/libcudnn* /usr/local/cuda/lib64
!chmod a+r /usr/local/cuda/lib64/libcudnn*

**Deep Speech Model Load, STT**

In [0]:
import deepspeech
from deepspeech import Model
import scipy.io.wavfile as wav

import glob
import webrtcvad
import logging
from timeit import default_timer as timer

In [0]:
def resolve_models(dirName):
    pb = glob.glob(dirName + "/*.pb")[0]
    logging.debug("Found Model: %s" % pb)

    lm = glob.glob(dirName + "/lm.binary")[0]
    trie = glob.glob(dirName + "/trie")[0]
    logging.debug("Found Language Model: %s" % lm)
    logging.debug("Found Trie: %s" % trie)

    return pb, lm, trie
    
def load_model(models, lm, trie):
    BEAM_WIDTH = 500
    LM_ALPHA = 0.75
    LM_BETA = 1.85

    model_load_start = timer()
    ds = Model(models, BEAM_WIDTH)
    model_load_end = timer() - model_load_start
    logging.debug("Loaded model in %0.3fs." % (model_load_end))

    lm_load_start = timer()
    ds.enableDecoderWithLM(lm, trie, LM_ALPHA, LM_BETA)
    lm_load_end = timer() - lm_load_start
    logging.debug('Loaded language model in %0.3fs.' % (lm_load_end))

    return [ds, model_load_end, lm_load_end]

def stt(ds, audio, fs):
    inference_time = 0.0
    audio_length = len(audio) * (1 / fs)

    # Run Deepspeech
    logging.debug('Running inference...')
    inference_start = timer()
    output = ds.stt(audio)
    inference_end = timer() - inference_start
    inference_time += inference_end
    logging.debug('Inference took %0.3fs for %0.3fs audio file.' % (inference_end, audio_length))

    return [output, inference_time]

In [0]:
output_graph, lm, trie = resolve_models('deepspeech-0.6.0-models')
output_graph, lm, trie

In [0]:
model_retval = load_model(output_graph, lm, trie)
model_retval 

In [0]:
fs, audio = wav.read("customdata/ttssample1.wav")
output = stt(model_retval[0], audio,fs)
print(output)

**LDC93S1 Test Script**

In [0]:
!pwd

In [0]:
!pip3 install $(python3 util/taskcluster.py --decoder)

In [0]:
%cd /content/deepspeech
!./bin/run-ldc93s1.sh